In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import Optional, Tuple, Collection
import re

from music21 import (
    converter,
    instrument,
    note,
    chord,
    pitch,
    interval,
    stream,
    volume,
    duration,
    midi,
    meter,
    key,
    tempo,
)
from music21.midi import MidiException
import pandas as pd

<IPython.core.display.Javascript object>

In [ ]:
# default_exp midi
# default_cls_lvl 3

<IPython.core.display.Javascript object>

# Parsing and representing MIDI notes

> Finding a compact representation of melody and rhythm.

Although MIDI notes have a lot of information, we want to distill them into a triplet -- namely pitch, duration and velocity:

In [ ]:
# export

Pitch, Duration, Velocity = str, float, int
Triplet = Tuple[Pitch, Duration, Velocity]

<IPython.core.display.Javascript object>

First we'll create some functions to turn single notes, chords and rests into these Triplets.

In [ ]:
# export


def _parse_duration(d: duration.Duration) -> Optional[str]:
    try:
        dur = float(d.quarterLength)
        name = d.type
        if dur > 0.0 and dur <= 32.0 and name != "inexpressible":
            return name
        else:
            return None
    except duration.DurationException:
        return None


def _parse_single_note(note: note.Note) -> Optional[Triplet]:
    dur = _parse_duration(note.duration)
    if dur is not None:
        return (str(note.pitch), dur, note.volume.velocity)
    else:
        return None


def _parse_chord(chord: chord.Chord) -> Optional[Triplet]:
    dur = _parse_duration(chord.duration)
    if dur is not None:
        return (".".join(str(n) for n in chord.normalOrder), dur, chord.volume.velocity)
    else:
        return None


def _parse_rest(rest: note.Rest) -> Optional[Triplet]:
    dur = _parse_duration(rest.duration)
    if dur is not None:
        return ("R", dur, 0)
    else:
        return None

<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq

quarter = duration.Duration("quarter")


def loud(n: note.Note, velocity=120) -> note.Note:
    n.volume = volume.Volume(velocity=velocity)
    return n


test_eq(
    ("C3", "quarter", 120), _parse_single_note(loud(note.Note("C3", duration=quarter)))
)
test_eq(None, _parse_single_note(note.Note("C3", duration=duration.Duration(34.0))))

test_eq(
    ("1.3.5", "quarter", 80),
    _parse_chord(loud(chord.Chord([1, 3, 5], duration=quarter), velocity=80)),
)
test_eq(None, _parse_chord(chord.Chord([1, 3, 5], duration=duration.Duration(0.0))))

test_eq(("R", "quarter", 0), _parse_rest(note.Rest(duration=quarter)))
test_eq(None, _parse_rest(note.Rest(duration=duration.Duration(0.0))))

<IPython.core.display.Javascript object>

As a normalization step, we need to transpose all the music to the same key, so we'll choose C for convenience.

In [ ]:
# export


def _transpose_to_C(s: stream.Stream) -> (stream.Stream, str):
    "Normalizes a stream to the C key, and returns the mode"
    k = s.analyze("key")
    i = interval.Interval(k.tonic, pitch.Pitch("C"))
    return s.transpose(i), k.mode

<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq

s = converter.parse("data/ff4-main.mid")
originalKey = s.analyze("key")

transposed, mode = _transpose_to_C(s)
transposedKey = transposed.analyze("key")

test_eq(key.Key("CM"), transposedKey)
test_eq(originalKey.mode, mode)

<IPython.core.display.Javascript object>

In [ ]:
# export


def _parse_part(elements, mode) -> Optional[dict]:
    d = {
        "notes": [],
        "instrument": "unknown",
        "bpm": 120,
        "time_signature": "4/4",
        "mode": mode,
    }

    for element in elements:
        if element.__class__ == note.Note:
            x = _parse_single_note(element)
            if x is not None:
                d["notes"].append(x)
        elif element.__class__ == chord.Chord:
            x = _parse_chord(element)
            if x is not None:
                d["notes"].append(x)
        elif element.__class__ == note.Rest:
            x = _parse_rest(element)
            if x is not None:
                d["notes"].append(x)
        elif isinstance(element, instrument.Instrument):
            if str(element) != "":
                d["instrument"] = str(element).lower()
        elif element.__class__ == tempo.MetronomeMark:
            d["bpm"] = element.number
        elif element.__class__ == meter.TimeSignature:
            d["time_signature"] = element.ratioString

    if len(d["notes"]) != 0:
        return d
    else:
        return None

<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq

s = converter.parse("data/rugrats.mid")
row = _parse_part(s.parts[0].recurse(), "major")

test_eq("piano", row["instrument"])
test_eq(102.0, row["bpm"])
test_eq("major", row["mode"])
test_eq("4/4", row["time_signature"])
test_eq(
    [("R", "eighth", 0), ("R", "breve", 0), ("C3", "eighth", 103)], row["notes"][0:3]
)

<IPython.core.display.Javascript object>

In [ ]:
# export


def parse_midi_file(file: str) -> Optional[pd.DataFrame]:
    """
    Attempts to parse a midi file into a Dataframe. Returns a Dataframe or None, together with the number of notes processed.
    """
    try:
        midi, mode = _transpose_to_C(converter.parse(file))
        parts = instrument.partitionByInstrument(midi)
        rows = []
        if parts:
            for part in parts:
                parsed = _parse_part(part.recurse(), mode)
                if parsed is not None:
                    rows.append(parsed)
        else:
            parsed = _parse_part(midi.flat.notes, mode)
            if parsed is not None:
                rows.append(parsed)

        for row in rows:
            row["pitches"] = [pitch for (pitch, dur, velocity) in row["notes"]]
            row["durations"] = [dur for (pitch, dur, velocity) in row["notes"]]
            row["velocities"] = [velocity for (pitch, dur, velocity) in row["notes"]]
            del row["notes"]

        return pd.DataFrame(rows)
    except MidiException:
        return None
    except IndexError:
        return None


def row_to_triplets(df: pd.DataFrame, row_index: int) -> Collection[Triplet]:
    """
    Takes a DataFrame at a specific row and turns that into a triplet.
    """
    pitches = df.get("pitches")[row_index]
    durations = df.get("durations")[row_index]
    velocities = df.get("velocities")[row_index]
    return list(zip(pitches, durations, velocities))

<IPython.core.display.Javascript object>

In [ ]:
!ls data

ff4-airship.mid ff4-main.mid    ff4-town.mid    midi.tar.gz     rugrats.mid


<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq, path

df = parse_midi_file(path("data/ff4-main.mid"))

test_eq(1, len(df))
test_eq("piano", df.get("instrument")[0])

triplets = row_to_triplets(df, 0)

test_eq(
    [("E5", "eighth", 110), ("B4", "eighth", 110), ("G4", "eighth", 110)], triplets[0:3]
)
test_eq(1, len(df))

df = parse_midi_file(path("data/rugrats.mid"))
test_eq(1, len(df))

<IPython.core.display.Javascript object>

## Turning triplets back into MIDI streams

Another interesting thing is to be able to go from triplets to MIDI streams, display their scores and even an audible IPython widget to play the audio.

But first of all, some utilities to turn these triplets into `music21` elements:

In [ ]:
# export


def triplet_to_note(triplet: Triplet):
    pitch, dur, vel = triplet
    dur = duration.Duration(dur)
    vol = volume.Volume(velocity=vel)
    if re.match(r"^[0-9]", pitch):
        c = chord.Chord([int(x) for x in pitch.split(".")], duration=dur)
        c.volume = vol
        return c
    elif pitch == "R":
        return note.Rest(duration=dur)
    else:
        n = note.Note(pitch, duration=dur)
        n.volume = vol
        return n


def triplets_to_stream(triplets: Collection[Triplet]) -> stream.Stream:
    s = stream.Stream()
    _ = [s.append(triplet_to_note(triplet)) for triplet in triplets]
    return s


def write_midi(stream, name):
    mf = midi.translate.streamToMidiFile(stream)
    mf.open(f"{name}.mid", "wb")
    mf.write()
    mf.close()

<IPython.core.display.Javascript object>

Now let's try to visualize and listen to these streams.

If you're running Mac or Linux, uncomment the right `brew` or `apt` call:

In [ ]:
# skip
# hide

#!brew install fluidsynth
#!apt install fluidsynth
#!wget ftp://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf3
#!pip install midi2audio

--2019-12-08 01:55:32--  ftp://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf3
           => ‘MuseScore_General.sf3’
Resolving ftp.osuosl.org (ftp.osuosl.org)... 140.211.166.134, 64.50.236.52, 64.50.233.100
Connecting to ftp.osuosl.org (ftp.osuosl.org)|140.211.166.134|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/musescore/soundfont/MuseScore_General ... done.
==> SIZE MuseScore_General.sf3 ... 39893918
==> PASV ... done.    ==> RETR MuseScore_General.sf3 ... done.
Length: 39893918 (38M) (unauthoritative)

MuseScore_General.s 100%[===================>]  38.04M  4.47MB/s    in 12s     

2019-12-08 01:55:48 (3.23 MB/s) - ‘MuseScore_General.sf3’ saved [39893918]



<IPython.core.display.Javascript object>

In [ ]:
# skip
# hide

%load_ext music21.ipython21.ipExtension

import json, random

from IPython.core.display import display, HTML, Javascript
from IPython.display import Audio

from midi2audio import FluidSynth


def show_score(score):
    xml = open(score.write("musicxml")).read()
    show_xml(xml)


def show_xml(xml):
    DIV_ID = "OSMD-div-" + str(random.randint(0, 1000000))
    # print("DIV_ID", DIV_ID)
    msg = "loading OpenSheetMusicDisplay"
    msg = ""
    display(
        HTML('<div style="background: white" id="' + DIV_ID + '">{}</div>'.format(msg))
    )

    # print('xml length:', len(xml))

    script = """
    console.log("loadOSMD()");
    function loadOSMD() { 
        return new Promise(function(resolve, reject){

            if (window.opensheetmusicdisplay) {
                console.log("already loaded")
                return resolve(window.opensheetmusicdisplay)
            }
            console.log("loading osmd for the first time")
            // OSMD script has a 'define' call which conflicts with requirejs
            var _define = window.define // save the define object 
            window.define = undefined // now the loaded script will ignore requirejs
            var s = document.createElement( 'script' );
            s.setAttribute( 'src', "https://cdn.jsdelivr.net/npm/opensheetmusicdisplay@0.3.1/build/opensheetmusicdisplay.min.js" );
            //s.setAttribute( 'src', "/custom/opensheetmusicdisplay.js" );
            s.onload=function(){
                window.define = _define
                console.log("loaded OSMD for the first time",opensheetmusicdisplay)
                resolve(opensheetmusicdisplay);
            };
            document.body.appendChild( s ); // browser will try to load the new script tag
        }) 
    }
    loadOSMD().then((OSMD)=>{
        console.log("loaded OSMD",OSMD)
        var div_id = "{{DIV_ID}}";
            console.log(div_id)
        window.openSheetMusicDisplay = new OSMD.OpenSheetMusicDisplay(div_id);
        openSheetMusicDisplay
            .load({{data}})
            .then(
              function() {
                console.log("rendering data")
                openSheetMusicDisplay.render();
              }
            );
    })
    """.replace(
        "{{DIV_ID}}", DIV_ID
    ).replace(
        "{{data}}", json.dumps(xml)
    )
    display(Javascript(script))
    return DIV_ID


def view_song(triplets, name="song"):
    stream = triplets_to_stream(triplets)
    write_midi(stream, f"{name}")

    FluidSynth("MuseScore_General.sf3").midi_to_audio(f"{name}.mid", f"{name}.wav")
    show_score(stream)
    return Audio(f"{name}.wav")

<IPython.core.display.Javascript object>

In [ ]:
# skip
triplets = row_to_triplets(df, 0)
view_song(triplets)